In [2]:
import pandas as pd

splits = {'train': 'ohsumed/train-00000-of-00001.parquet', 'test': 'ohsumed/test-00000-of-00001.parquet'}
training = pd.read_parquet("hf://datasets/community-datasets/ohsumed/" + splits["train"])
test = pd.read_parquet("hf://datasets/community-datasets/ohsumed/" + splits["test"])

C:\Users\asus9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
training['title_abstract'] = training['title'] + ' ' + training['abstract']
test['title_abstract'] = test['title'] + ' ' + test['abstract']

In [6]:
from datasets import Dataset
training.head()
test.head()
train_dataset = Dataset.from_pandas(training)
test_dataset = Dataset.from_pandas(test)

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
training.isna().sum()

,0
seq_id,0
medline_ui,0
mesh_terms,0
title,0
publication_type,0
abstract,0
author,0
source,0
title_abstract,0


In [ ]:
test.isna().sum()

,0
seq_id,0
medline_ui,0
mesh_terms,0
title,0
publication_type,0
abstract,0
author,0
source,0
title_abstract,0


In [ ]:
training.shape

(54709, 9)

In [ ]:
test.shape

(293855, 9)

In [ ]:
import pandas as pd

def is_sensitive(mesh_terms):
  if isinstance(mesh_terms, str):
    return 1 if 'urogenital' in mesh_terms.lower() or 'pregnancy complications' in mesh_terms.lower() else 0
  else:
    return 0

training['sensitivity'] = training['mesh_terms'].apply(is_sensitive)

sensitivity_count = training['sensitivity'].sum()
print(f"Numero di documenti sensibili in training (sensitivity = 1): {sensitivity_count}")

test['sensitivity'] = test['mesh_terms'].apply(is_sensitive)

sensitivity_count = test['sensitivity'].sum()
print(f"Numero di documenti sensibili in test (sensitivity = 1): {sensitivity_count}")

Numero di documenti sensibili in training (sensitivity = 1): 605
Numero di documenti sensibili in test (sensitivity = 1): 2945


In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(training, test_size=0.15, random_state=23)  # 15% per la convalida

print(train_data.shape)
print(val_data.shape)
print(test.shape)

(46502, 10)
(8207, 10)
(293855, 10)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = training['title_abstract']
y_train = training['sensitivity']
X_test = test['title_abstract']
y_test = test['sensitivity']

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X_test_vec = vectorizer.transform(X_test)
y_pred = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9901
Precision: 0.5478
Recall: 0.0934
F1-score: 0.1596
